In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
from game.hyperparams import Hp

import pickle
import glob
import os


import glob
import os
import pickle
import sys
import inspect

sys.path.append(os.getcwd())
sys.path.append("")
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0, parentdir) 

import torch

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import numpy as np
from pandas import DataFrame


from game.hint_play_game import TwoRoundHintGame
from train_qlearner import obs_to_agent

from game.hyperparams import Hp
from agent.qlearner import QLearner

GPU not available, CPU used


In [2]:
hp_default = Hp(hand_size=5,
                          nlab1=3,
                          nlab2=3,
                          )

In [55]:
# hand1 = [np.array([0., 1., 0., 1., 0., 0.]),
#          np.array([0., 1., 0., 0., 1., 0.]),
#          np.array([1., 0., 0., 0., 0., 1.]),
#          np.array([1., 0., 0., 0., 0., 1.]),
#          np.array([1., 0., 0., 0., 0., 1.]),]

hand1 = [np.array([0., 1., 0., 1., 0., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 1., 0., 0., 1., 0.]),]
         
hand2 = [np.array([0., 1., 0., 0., 1., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),
         np.array([0., 0., 1., 1., 0., 0.]),]


def get_given_initial_state(hp=hp_default, hand1=hand1, hand2=hand2, playable_card_num=4):
    """
    Generate initial obs assignment
    P1 gets all hands as well as the playable card for P2
    P2 gets all hands as well as an empty card
    """
    info = {}
    o1=hand1 
    o2=hand2
    o1.append(o2[playable_card_num])
    o2.append(np.zeros((hp.nlab1 + hp.nlab2)))
    o1 = np.array(o1).flatten()
    o2 = np.array(o2).flatten()
    info['playbale_card_num'] = playable_card_num
    return (o1, o2), info

In [56]:
obs, info = get_given_initial_state()
initial_config = {'obs': obs, 'info':info}

In [51]:
agent1s = []
agent2s = []
agent_paths = sorted(glob.glob(os.path.join("res/Att3_hs_5_l1_3_l2_3_FalseFalse4000000", "*.pkl")))
cluster1 = [agent_paths[i] for i in [4,6,10,12,13,17]]
cluster2 = [agent_paths[i] for i in  [1,2,3,5,7,8,9,11,14,15,16,]]

for filename in cluster1:
       with open(filename, "rb") as f:
               res = pickle.load(f)
               agent1s += [res['p1']]
               agent2s += [res['p2']]

In [57]:
p1 = agent1s[2]
p2 = agent2s[2]

In [60]:
def sample_games(p1, p2, episodes=10, verbose=False, initial_config=initial_config):
    hp1 = p1.hp
    hp2 = p2.hp
    env = TwoRoundHintGame(hp=hp1, initial_config=initial_config)

    hints = []
    plays = []

    print(f'agents are {hp1.agent_type} and {hp2.agent_type}')

    for i_episode in range(episodes):
        # Initialize the environment and state
        obs1, info = env.reset()
        if verbose:
            env.render()
        obs1_a1, obs1_a2 = obs_to_agent(obs1, hp=hp1)
        if hp1.agent_type != 'FF':
            obs1_a1 = obs1_a1.reshape(-1, hp1.nlab1 + hp1.nlab2).T
        # P1 select and perform a hint
        # obs1_a1 = torch.tensor(obs1_a1, device=device)
        a1 = p1.select_action(torch.tensor([obs1_a1], device=device), evaluate=True)
        obs2, _, _, _ = env.step(a1)
        hint_card = env.get_card_symbol_in_round()
        if verbose:
            env.render()
        # P2 plays a card
        obs2_a1, obs2_a2 = obs_to_agent(obs2, hp=hp2)
        if hp2.agent_type != 'FF':
            obs2_a2 = obs2_a2.reshape(-1, hp2.nlab1 + hp2.nlab2).T
        a2 = p2.select_action(torch.tensor([obs2_a2], device=device), evaluate=True)
        _, r, _, _ = env.step(a2)
        r = torch.tensor([r[0]], device=device)
        rewards.append(r.numpy()[0])
        play_card = env.get_card_symbol_in_round()
        if verbose:
            env.render()
        hints.append(hint_card)
        plays.append(play_card)
    return hints, plays

In [61]:
sample_games(p1, p2, verbose=False)

agents are Att3 and Att3


AttributeError: 'TwoRoundHintGame' object has no attribute 'get_card_symbol_in_round'